In [1]:
# Modifying the path so we can import from src directory.
import sys
import os
sys.path.append(os.path.abspath('..'))

import copy
import pickle
import time

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from src.example_graphs import simple_undirected_graph, simple_directed_graph
from src.UndirectedGraph import UndirectedGraph
from src.DirectedGraph import DirectedGraph
from src.DataLoader import DataLoader
from src.GraphCreator import GraphCreator

from src.io_helpers import pickle_obj, load_pickled_obj
from src.networkx_helpers import combine_graphs

In [2]:
G = nx.MultiDiGraph()
G.add_edge(1, 2, key=None, attr={
    'property1': 'something here',
    'property2': 2,
    'property3': [1, 2, 3, 'hello', lambda x: x**2],
    4: 'something here',
})
G.add_edge(1, 2, key=None, attr={
    'property2': 5
})

1

In [3]:
G[1]

AdjacencyView({2: {0: {'attr': {'property1': 'something here', 'property2': 2, 'property3': [1, 2, 3, 'hello', <function <lambda> at 0xa20b6b7a0>], 4: 'something here'}}, 1: {'attr': {'property2': 5}}}})

In [6]:
list(G.nodes_with_selfloops())

[]

In [22]:
for node in G:
    for node2 in G[node]:
        for edge in G[node][node2]:
            print(node, node2, G[node][node2][edge])

1 2 {'attr': {'property1': 'something here', 'property2': 2, 'property3': [1, 2, 3, 'hello', <function <lambda> at 0xa20b6b7a0>], 4: 'something here'}}
1 2 {'attr': {'property2': 5}}


In [30]:
# 1 is the node we're looking at
# 2 is the node that 1 points to
# 0 is the first edge
# 'attr' gets the attribute dict
attr_dict_1 = G[1][2][0]['attr']
attr_dict_1

{'property1': 'something here',
 'property2': 2,
 'property3': [1, 2, 3, 'hello', <function __main__.<lambda>(x)>],
 4: 'something here'}

In [32]:
attr_dict_1['property2']

2

In [31]:
# 1 is the node we're looking at
# 2 is the node that 1 points to
# 1 is the second edge
# 'attr' gets the attribute dict
attr_dict_2 = G[1][2][1]['attr']
attr_dict_2

{'property2': 5}

In [33]:
attr_dict_2['property2']

5

In [34]:
attr_dict_1['property2'] + attr_dict_2['property2']

7

In [44]:
def get_edge_attrs(graph, node1, node2):
    return dict(graph[node1][node2].items())

In [47]:
get_edge_attrs(G, 1, 2)

{0: {'attr': {'property1': 'something here',
   'property2': 2,
   'property3': [1, 2, 3, 'hello', <function __main__.<lambda>(x)>],
   4: 'something here'}},
 1: {'attr': {'property2': 5}}}

In [70]:
def sum_numeric_properties(attr_dict, numeric_attrs):
    summed_dict = {attr: 0 for attr in numeric_attrs}
    for edge in attr_dict:
        for attr in numeric_attrs:
            summed_dict[attr] += attr_dict[edge]['attr'][attr]
    
    return summed_dict

In [71]:
sum_numeric_properties(get_edge_attrs(G, 1, 2), ['property2'])

{'property2': 7}

In [72]:
def aggregate_numeric_properties(attr_dict, numeric_attrs, agg_func):
    aggregated_dict = {attr: [] for attr in numeric_attrs}
    for edge in attr_dict:
        for attr in numeric_attrs:
            aggregated_dict[attr].append(attr_dict[edge]['attr'][attr])
    
    for k in aggregated_dict:
        aggregated_dict[k] = agg_func(aggregated_dict[k])
    
    return aggregated_dict

In [78]:
aggregate_numeric_properties(get_edge_attrs(G, 1, 2), ['property2'], np.median)

{'property2': 3.5}

In [68]:
def count_edges(graph, node1, node2):
    return len(graph[node1][node2])

In [69]:
count_edges(G, 1, 2)

2